In [ ]:
import os
import glob
import numpy as np
from tensorflow.keras.utils import  plot_model
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import keras as K
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from PIL import Image, ImageDraw
import pandas as pd
import seaborn as sns
from keras.callbacks import ModelCheckpoint
import keras.callbacks
import tensorflow.keras.losses
import os
import pydot


In [ ]:
# Image Parameters
N_CLASSES = 2 # total number of classes
IMG_HEIGHT = 448 # the image height to be resized to
IMG_WIDTH = 448 # the image width to be resized to
CHANNELS = 3 # The 3 color channels, change to 1 if grayscale
initial_train_data = r'C:\Users\densh\Desktop\catdogs\dataset\augmented_data'
initial_validation = r'C:\Users\densh\Desktop\catdogs\dataset\valid'
tf_train_data = r'C:\Users\densh\Desktop\catdogs\dataset\augmented_data_tf'
tf_train = r'C:\Users\densh\Desktop\catdogs\dataset\train_tf'
tf_validation = r'C:\Users\densh\Desktop\catdogs\dataset\valid_tf'

In [ ]:
#uncomment to plot nn graph
os.environ["PATH"] += os.pathsep + r'C:\Program Files\Graphviz\bin'
os.environ["PATH"] += os.pathsep + r'C:\Users\densh\AppData\Roaming\Python\Python38\Scripts'

In [ ]:
from keras.utils import to_categorical
def parse_dataset(dataset_path, ext='jpg'):
    """
    Used to extract information about our dataset. It does iterate over all images and return a DataFrame with
    the data (age, gender and sex) of all files.
    """
    def parse_info_from_file(path):
        """
        Parse information from a single file
        """
        try:
            title, ext = os.path.splitext(os.path.basename(path))
            txt_file_name = title + '.txt'
            with open(os.path.join(os.path.dirname(os.path.abspath(path)),txt_file_name),'r') as old_txt:
                labelList = old_txt.readlines()
                for label in labelList:
                    label = label.strip().split()
                    class_= int(label[1])
                    x1    = float(label[2])
                    y1    = float(label[3])
                    x2    = float(label[4])
                    y2    = float(label[5])
                    
                    w = x2 - x1
                    h = y2 - y1
                    x1 = x1+w/2
                    y1 = y1+h/2
                    x2=w
                    y2=h
            return class_, x1, y1, x2, y2
        except Exception as ex:
            return class_, None, None, None, None
        
    files = glob.glob(os.path.join(dataset_path, "*.%s" % ext))
    
    records = []
    for file in files:
        info = parse_info_from_file(file)
        records.append(info)
        
    df = pd.DataFrame(records)
    df['file'] = files
    df.columns = ['class', 'x1', 'y1', 'x2', 'y2', 'file']
    df = df.dropna()
    
    return df

df = parse_dataset(tf_train)
df.head()

In [ ]:
val_df = parse_dataset(tf_validation)
val_df.head()

In [ ]:
class DataGenerator():
    """
    Data generator for the UTKFace dataset. This class should be used when training our Keras multi-output model.
    """
    def __init__(self, df):
        self.df = df
        
    def generate_split_indexes(self):
        p = np.random.permutation(len(self.df))
        train_idx = p[:]
        
        return train_idx
    
    def generate_test_split_indexes(self,length):
        p = np.random.randint(0,len(self.df),length)
        train_idx = p[:]
        
        return train_idx
    
    def preprocess_image(self, img_path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        
        im = Image.open(img_path)
        im = im.resize((IMG_WIDTH, IMG_HEIGHT))
        im = np.array(im) / 255.0
        return im
        
    def generate_images(self, image_idx, is_training, batch_size):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        
        # arrays to store our batched data
        images, classes, x1, y1, x2, y2 = [], [], [], [], [], []
        while True:
            for idx in image_idx:
                imag = self.df.iloc[idx]
                
                classes_ = imag['class']
                x1_ = imag['x1']
                y1_ = imag['y1']
                x2_ = imag['x2']
                y2_ = imag['y2']
                file = imag['file']
                im = self.preprocess_image(file)
                
                classes.append(classes_)
                x1.append(x1_)
                y1.append(y1_)
                x2.append(x2_)
                y2.append(y2_)
                images.append(im)
                # yielding condition
                if len(images) >= batch_size:
                    yield np.array(images), [np.array(to_categorical(classes,2)),np.array(x1),np.array(y1), np.array(x2), np.array(y2)]
                    images, classes, x1, y1, x2, y2 = [], [], [], [], [], []
                    
            if not is_training:
                break
                
data_generator = DataGenerator(df)
train_idx = data_generator.generate_split_indexes() 
v_data_generator = DataGenerator(val_df)
valid_idx = v_data_generator.generate_split_indexes() 

In [ ]:
import tensorflow.keras.applications

inputs = keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, CHANNELS), name="img_input")

N_mobile = tensorflow.keras.applications.MobileNetV2(input_shape=(IMG_WIDTH, IMG_HEIGHT, CHANNELS), include_top=False, weights='imagenet', alpha = 0.35)#, weights='imagenet')
N_mobile.trainable = False
base_model_output = N_mobile.layers[-1].output
#class_output = layers.Conv2D(256, (2,2), activation="relu")(base_model_output)
#d = layers.Conv2D(1024, (2,2), activation="relu")(base_model_output)
class_output = layers.GlobalAveragePooling2D()(base_model_output)
class_output = layers.BatchNormalization()(class_output)
class_output = layers.Dense(2, activation="softmax",name="class_output")(class_output)

x = layers.Conv2D(64, kernel_size=3)(base_model_output)
#x = layers.Conv2D(4, activation="sigmoid", kernel_size=3)(x)
x = layers.Conv2D(4, kernel_size=3)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Flatten()(x)
box_output1 = layers.Dense(1, activation="linear", name= "Box_output1")(x)

x = layers.Conv2D(64, kernel_size=3)(base_model_output)
#x = layers.Conv2D(4, activation="sigmoid", kernel_size=3)(x)
x = layers.Conv2D(4, kernel_size=3)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Flatten()(x)
box_output2 = layers.Dense(1, activation="linear", name= "Box_output2")(x)

x = layers.Conv2D(64, kernel_size=3)(base_model_output)
#x = layers.Conv2D(4, activation="sigmoid", kernel_size=3)(x)
x = layers.Conv2D(2, kernel_size=3)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Flatten()(x)
box_output3 = layers.Dense(1, activation="relu", name= "Box_output3")(x)

x = layers.Conv2D(64, kernel_size=3)(base_model_output)
#x = layers.Conv2D(4, activation="sigmoid", kernel_size=3)(x)
x = layers.Conv2D(2, kernel_size=3)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Flatten()(x)
box_output4 = layers.Dense(1, activation="relu", name= "Box_output4")(x)

model2 = keras.Model(inputs=N_mobile.input, outputs = [class_output,box_output1,box_output2,box_output3,box_output4], name="cats_dogs")


In [ ]:
plot_model(model2,to_file='model.pdf',show_shapes=True)

In [ ]:
def log_mse(y_true, y_pred):
    return tf.reduce_mean(tf.math.log1p(tf.math.squared_difference(y_pred, y_true)), axis=-1)

def focal_loss(alpha=0.9, gamma=2):
  def focal_loss_with_logits(logits, targets, alpha, gamma, y_pred):
    weight_a = alpha * (1 - y_pred) ** gamma * targets
    weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)
    
    return (tf.math.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(-logits)) * (weight_a + weight_b) + logits * weight_b

  def loss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
    logits = tf.math.log(y_pred / (1 - y_pred))

    loss = focal_loss_with_logits(logits=logits, targets=y_true, alpha=alpha, gamma=gamma, y_pred=y_pred)

    return tf.reduce_mean(loss)

  return loss

In [ ]:
init_lr = 1e-2
epochs = 50
model2.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-2, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),#SGD(lr = 1e-2),#Adam(lr=init_lr,beta_1=0.9),
     loss={
                  'Box_output1': log_mse,
                  'Box_output2': log_mse,
                  'Box_output3': log_mse,
                  'Box_output4': log_mse,
                  'class_output': focal_loss()},
     loss_weights={
                  'Box_output1': 1,
                  'Box_output2': 1,
                  'Box_output3': 1,
                  'Box_output4': 1,
                  'class_output': 1},
     metrics={
                  "Box_output1":  'mse',
                  "Box_output2":  'mse',
                  "Box_output3":  'mse',
                  "Box_output4":  'mse',
                  "class_output": 'accuracy',
    },
)

In [ ]:
def scheduler(epoch, lr):
    if epoch>20:
        return lr*0.9
    else:
        return lr
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.0001)

In [ ]:
batch_size = 128
valid_batch_size = 64
train_gen =   data_generator.generate_images(train_idx, is_training=True, batch_size=batch_size)
valid_gen = v_data_generator.generate_images(valid_idx, is_training=True, batch_size=valid_batch_size)
callbacks = [
    #ModelCheckpoint(r"C:\Users\densh\Desktop\catdogs\keras_try\Checkpoint", monitor='val_loss', save_best_only=True), 
    reduce_lr, callback,
    keras.callbacks.TensorBoard(
    log_dir=r"C:\Users\densh\Desktop\catdogs\keras_try\log",write_images=True,
    histogram_freq=0,  # How often to log histogram visualizations
    embeddings_freq=0,  # How often to log embedding visualizations
    update_freq="epoch")  # How often to write logs (default: once per epoch)
]

In [ ]:
model2.fit(train_gen,steps_per_epoch=len(train_idx)//(1*batch_size),  epochs=100, callbacks=callbacks, shuffle=True, validation_data=valid_gen, validation_steps=len(valid_idx)//valid_batch_size)

In [ ]:
model2.save(r"C:\Users\densh\Desktop\catdogs\keras_try\model_testing\model5hw", overwrite=True, include_optimizer=False, save_format='tf', signatures=None, options=None, save_traces=False)


In [ ]:
train_idx = v_data_generator.generate_test_split_indexes(10)
test_gen  = v_data_generator.generate_images(train_idx, is_training=False, batch_size=1)
c = model2.predict(test_gen,steps=len(train_idx))
c

In [ ]:
df2=0
df2 = val_df.iloc[train_idx,]
df2.drop_duplicates()

In [ ]:
import cv2
font = cv2.FONT_HERSHEY_SIMPLEX
# fontScale
fontScale = 1
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2 
# Using cv2.putText() method
i=0
for idx in train_idx:
    file = val_df.iloc[int(idx)]
    for image in glob.glob(os.path.join(file['file'])):
        title, ext = os.path.splitext(os.path.basename(image))
        txt_file_name = title + '.txt'
        cv_image = cv2.imread(image)
        H, W, _ = cv_image.shape
        #top_left = (int(c[1][i]*W),int(c[2][i]*H))
        top_left = (int((c[1][i]-c[3][i]/2)*W),int((c[2][i]+c[4][i]/2)*H))
        #bottom_right = (int(c[3][i]*W),int(c[4][i]*H))
        bottom_right = (int((c[1][i]+c[3][i]/2)*W),int((c[2][i]-c[4][i]/2)*H))
        image = cv2.rectangle(cv_image, top_left, bottom_right, [0, 0, 0], 2)
        
        top_left = (int((df2['x1'][idx]+df2['x2'][idx]/2)*W),int((df2['y1'][idx]-df2['y2'][idx]/2)*H))
        bottom_right = (int((df2['x1'][idx]-df2['x2'][idx]/2)*W),int((df2['y1'][idx]+df2['y2'][idx]/2)*H))
        #top_left = (int(df2['x1'][idx]*W), int(df2['y1'][idx]*H))
        #bottom_right = (int(df2['x2'][idx]*W), int(df2['y2'][idx]*H))
        image = cv2.rectangle(image, top_left, bottom_right, [255, 0, 0], 2)

        if float(c[0][i][0])>float(c[1][i][0]):
            image = cv2.putText(image, 'Cat', (int(c[1][i]*W),int(c[1][i]*H)), font, fontScale, color, thickness, cv2.LINE_AA)
        else:
            image = cv2.putText(image, 'Dog', (int(c[1][i]*W),int(c[1][i]*H)), font, fontScale, color, thickness, cv2.LINE_AA)
        i=i+1
        #cv2.imshow('img',image)
        cv2.imwrite(r'C:\Users\densh\Desktop\catdogs\keras_try\testing/'+title+'.jpg',image)
        
        print(idx)